<h1>Segmenting Neighborhoods with Latitude and Longtitude Coordinates</h1>

In [8]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 14.3MB/s ta 0:00:01


In [9]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

In [10]:
URL = "https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

In [13]:
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1


In [15]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighbors = pd.DataFrame(columns=column_names)

lat_lng_coords = None

for data in range(0, len(neighborhoods)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,'Borough': borough,'Neighborhood': neighborhood_name,'Latitude': lat_lng_coords[0],
                                  'Longitude': lat_lng_coords[1]}, ignore_index=True)
neighbors


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286
5,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
6,M9A,Etobicoke,Islington Avenue,43.662299,-79.528195
7,M1B,Scarborough,Rouge,43.811525,-79.195517
8,M1B,Scarborough,Malvern,43.811525,-79.195517
9,M3B,North York,Don Mills North,43.749055,-79.362227


In [18]:
neighbors.shape

(209, 5)

<h1>Clustering the neighborhoods in Toronto</h1>

In [22]:
!{sys.executable} -m pip install folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 


     |████████████████████████████████| 92kB 5.6MB/s eta 0:00:011


In [23]:
VERSION = '20180605'

neighborhood_name = neighbors.loc[0, 'Neighborhood'] 
neighborhood_latitude = neighbors.loc[0, 'Latitude'] 
neighborhood_longitude = neighbors.loc[0, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))
radius = 500 
LIMIT = 100 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()

Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
2,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
3,Grover Pub and Grub,Pub,43.679181,-79.297215
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng,v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [45]:
toronto_venues = getNearbyVenues(names=neighborhoods,latitudes=neighbors['Latitude'],longitudes=neighbors['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North

Woodbine Gardens
Parkview Hill
Ryerson

Garden District

Glencairn

Cloverdale

Islington
Martin Grove

Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens

Eringate

Markland Wood
Old Burnhamthorpe

Guildwood

Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor
Downsview North

Wilson Heights
Thorncliffe Park
Adelaide

King

Richmond

Dovercourt Village
Dufferin

Scarborough Village
Fairview

Henry Farm
Oriole

Northwood Park
York University
East Toronto
Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park

Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East

The Danforth West

Rive

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1045, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Parkwoods,43.75242,-79.329242,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,Parkwoods,43.75242,-79.329242,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,Parkwoods,43.75242,-79.329242,Grover Pub and Grub,43.679181,-79.297215,Pub
4,Parkwoods,43.75242,-79.329242,Upper Beaches,43.680563,-79.292869,Neighborhood


In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,5,5,5,5,5,5
Agincourt,5,5,5,5,5,5
Agincourt North,5,5,5,5,5,5
Albion Gardens,5,5,5,5,5,5
Alderwood,5,5,5,5,5,5
Bathurst Manor,5,5,5,5,5,5
Bathurst Quay,5,5,5,5,5,5
Bayview Village,5,5,5,5,5,5
Beaumond Heights,5,5,5,5,5,5


In [31]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Trail,Health Food Store,Neighborhood,Other Great Outdoors,Pub
0,1,0,Parkwoods,0,0
1,0,0,Parkwoods,1,0
2,0,1,Parkwoods,0,0
3,0,0,Parkwoods,0,1
4,0,0,Parkwoods,0,0


In [32]:
toronto_onehot.shape

(1045, 5)

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape


(208, 5)

In [36]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [38]:
# sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 3
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Adelaide,Pub,Other Great Outdoors,Health Food Store
1,Agincourt,Pub,Other Great Outdoors,Health Food Store
2,Agincourt North,Pub,Other Great Outdoors,Health Food Store
3,Albion Gardens,Pub,Other Great Outdoors,Health Food Store
4,Alderwood,Pub,Other Great Outdoors,Health Food Store


<h2>K-Means Clustering</h2>

In [41]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,0,Pub,Other Great Outdoors,Health Food Store
1,M4A,North York,Victoria Village,43.730600,-79.313265,0,Pub,Other Great Outdoors,Health Food Store
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0,Pub,Other Great Outdoors,Health Food Store
3,M6A,North York,Lawrence Heights,43.723270,-79.451286,0,Pub,Other Great Outdoors,Health Food Store
4,M6A,North York,Lawrence Manor,43.723270,-79.451286,0,Pub,Other Great Outdoors,Health Food Store


<h2>Map with Clusters</h2>

In [43]:
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,0,Pub,Other Great Outdoors,Health Food Store
1,North York,0,Pub,Other Great Outdoors,Health Food Store
2,Downtown Toronto,0,Pub,Other Great Outdoors,Health Food Store
3,North York,0,Pub,Other Great Outdoors,Health Food Store
4,North York,0,Pub,Other Great Outdoors,Health Food Store
5,Downtown Toronto,0,Pub,Other Great Outdoors,Health Food Store
6,Etobicoke,0,Pub,Other Great Outdoors,Health Food Store
7,Scarborough,0,Pub,Other Great Outdoors,Health Food Store
8,Scarborough,0,Pub,Other Great Outdoors,Health Food Store
9,North York,0,Pub,Other Great Outdoors,Health Food Store
